In [1]:
import ROOT
from os import path
import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
import numpy as np
from statistics import mean, median
import math
from array import array
import csv
from datetime import datetime
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from keras import backend as bck

def r2(ytrue, ypred):
    print(type(ytrue))
    print(type(ypred))
    res = bck.sum(bck.square(ytrue-ypred))
    tot = bck.sum(bck.square(ytrue-bck.mean(ytrue)))
    return 1-res/tot

def Build_ML_Feature_Arrays_ptTrue(
    input_file_path, input_tree_name, pt_true_min, pt_true_max) :
    """
    Creates an array of all features exported from ROOT.
    Applies a cut using pT_True values between pt_true_min and pt_true_max.
    """

    input_file = None;
    if (ROOT.gSystem.AccessPathName(input_file_path)) :
        print("Input file path does not exist:", input_file)
        exit()
    else :
        input_file = ROOT.TFile.Open(input_file_path, "READ")
        print("Input file accessed successfully. Output file generated.")
    
    print("Accessing input tree...")
    input_tree = input_file.Get(input_tree_name)
    print("Input tree accessed successfully.")
    
    # Setup Arrays
    X_values_A  = []  # Array of arrays of inputs corresponding to pT_true as PYTHIA jet pT
    y_values_A  = []  # Array of targets for regression, pT_true is PYTHIA jet pT
    
    X_values_B  = []  # Array of arrays of inputs corresponding to pT_true as jet pT * PYTHIA pT / const. pT
    y_values_B  = []  # Array of targets for regression, pT_true is jet pT * PYTHIA pT / const. pT

    # Predictors
    jet_pt_raw       = None  # Raw/uncorrected jet pt
    jet_pt_corr      = None  # Corrected jet pt
    jet_mass         = None
    jet_area         = None
    jet_area_err     = None
    jet_const_n      = None
    const_pt_mean    = None  # Mean pt of jet constituents
    const_pt_median  = None  # Mean pt of jet constituents
    const_1_pt       = None  # pt of jet constituent particle 1
    const_2_pt       = None  # pt of jet constituent particle 2
    const_3_pt       = None  # pt of jet constituent particle 3
    const_4_pt       = None  # pt of jet constituent particle 4
    const_5_pt       = None  # pt of jet constituent particle 5
    const_6_pt       = None  # pt of jet constituent particle 6
    const_7_pt       = None  # pt of jet constituent particle 7
    const_8_pt       = None  # pt of jet constituent particle 8
    const_9_pt       = None  # pt of jet constituent particle 9
    const_10_pt      = None  # pt of jet constituent particle 10
    jet_y            = None
    jet_phi          = None
    jet_rho          = None

    # Targets
    jet_pt_true_A    = None  # True jet pt (determined from PYTHIA jets)
    jet_pt_true_B    = None

    # Helper Variables
    event_counter    = 0
    event_n_total    = 20000
    jet_n            = None  # Number of jets in an event
    jet_n_counter_A  = 0
    jet_n_counter_B  = 0
    jet_const_pt_arr = []    # Array of jet constituents and their values
    sc_correction_arr_A = []
    sc_correction_arr_B = []
    
    print("Preparing to collect data from tree...")

    # Collecting from TTree
    for event in input_tree :  
        jet_n = event.jet_n

        for jet in range(0, 2) :
            jet_pt_raw      = input_tree.jet_pt_raw[jet]
            jet_pt_corr     = input_tree.jet_pt_corr[jet]
            jet_mass        = input_tree.jet_mass[jet]
            jet_area        = input_tree.jet_area[jet]
            jet_const_n     = input_tree.jet_const_n[jet]
            const_pt_mean   = input_tree.const_pt_mean[jet]
            const_pt_median = input_tree.const_pt_median[jet]
            const_1_pt      = input_tree.const_1_pt[jet]
            const_2_pt      = input_tree.const_2_pt[jet]
            const_3_pt      = input_tree.const_3_pt[jet]
            const_4_pt      = input_tree.const_4_pt[jet]
            const_5_pt      = input_tree.const_5_pt[jet]
            const_6_pt      = input_tree.const_6_pt[jet]
            const_7_pt      = input_tree.const_7_pt[jet]
            const_8_pt      = input_tree.const_8_pt[jet]
            const_9_pt      = input_tree.const_9_pt[jet]
            const_10_pt     = input_tree.const_10_pt[jet]
            jet_y           = input_tree.jet_y[jet]
            jet_phi         = input_tree.jet_phi[jet]
            jet_rho         = input_tree.jet_rho[jet]
            
            jet_pt_true_A   = input_tree.jet_pt_true_pythia[jet]
            jet_pt_true_B   = input_tree.jet_pt_true_paper[jet]
            
            temp_jet_arr = [
                    jet_pt_corr,      jet_pt_raw,       jet_area, jet_mass, 
                        jet_const_n,     const_pt_mean,   const_pt_median, jet_rho,
                    const_1_pt,      const_2_pt,      const_3_pt,      const_4_pt,
                    const_5_pt,      const_6_pt,      const_7_pt,      const_8_pt,
                    const_9_pt,      const_10_pt,     jet_y,           jet_phi]
            
            if (jet_pt_true_A != 0.0) and (jet_pt_true_A > pt_true_min) and (jet_pt_true_A < pt_true_max) :
                X_values_A.append(temp_jet_arr)

                y_values_A.append(jet_pt_true_A)
                
                sc_correction_arr_A.append(jet_pt_corr)
                
                jet_n_counter_A   = jet_n_counter_A + 1
                
                if event_counter % 1000 == 0 : print(f"Event: {event_counter:3.0f} | Jet: {jet:2.0f} | pTraw: {jet_pt_raw:3.3f} | pTcorr: {jet_pt_corr: 3.3f} | pTtrue_A: {jet_pt_true_A: 5.3f}")
            
            if (jet_pt_true_B != 0.0) and (jet_pt_true_B > pt_true_min) and (jet_pt_true_B < pt_true_max) :
                X_values_B.append(temp_jet_arr)
                
                y_values_B.append(jet_pt_true_B)

                sc_correction_arr_B.append(jet_pt_corr)
                
                jet_n_counter_B   = jet_n_counter_B + 1

                if event_counter % 1000 == 0 : print(f"Event: {event_counter:3.0f} | Jet: {jet:2.0f} | pTraw: {jet_pt_raw:3.3f} | pTcorr: {jet_pt_corr: 3.3f} | pTtrue_B: {jet_pt_true_B: 5.3f}")
            
        event_counter += 1

    print(f"All data transferred to array. Testing with {jet_n_counter_A} A-jets and {jet_n_counter_B} B-jets.\n")
    print(f"Training set A: {len(X_values_A)} / {len(y_values_A)} / {len(sc_correction_arr_A)}")
    print(f"Training set B: {len(X_values_B)} / {len(y_values_B)} / {len(sc_correction_arr_B)}")

    input_file.Close()
    print("Input file closed.")
    
    return X_values_A, y_values_A, sc_correction_arr_A, X_values_B, y_values_B, sc_correction_arr_B

    

def Build_ML_Feature_Arrays_ptCorr(
    input_file_path, input_tree_name, pt_corr_min, pt_corr_max) :
    """
    WARNING: CODE MAY BE OUTDATED
    Creates an array of all features exported from ROOT.
    Applies a cut using pT_Corrected values between pt_corr_min and pt_corr_max.
    This was used once for an alternative selection method and may no longer work.
    """

    input_file = None;
    if (ROOT.gSystem.AccessPathName(input_file_path)) :
        print("Input file path does not exist:", input_file)
        exit()
    else :
        input_file = ROOT.TFile.Open(input_file_path, "READ")
        print("Input file accessed successfully. Output file generated.")
    
    print("Accessing input tree...")
    input_tree = input_file.Get(input_tree_name)
    print("Input tree accessed successfully.")

    # Setup Arrays
    X_values_A  = []  # Array of arrays of inputs corresponding to pT_true as PYTHIA jet pT
    y_values_A  = []  # Array of targets for regression, pT_true is PYTHIA jet pT
    
    X_values_B  = []  # Array of arrays of inputs corresponding to pT_true as jet pT * PYTHIA pT / const. pT
    y_values_B  = []  # Array of targets for regression, pT_true is jet pT * PYTHIA pT / const. pT

    # Predictors
    jet_pt_raw       = None  # Raw/uncorrected jet pt
    jet_pt_corr      = None  # Corrected jet pt
    jet_mass         = None
    jet_area         = None
    jet_area_err     = None
    jet_const_n      = None
    const_pt_mean    = None  # Mean pt of jet constituents
    const_pt_median  = None  # Mean pt of jet constituents
    const_1_pt       = None  # pt of jet constituent particle 1
    const_2_pt       = None  # pt of jet constituent particle 2
    const_3_pt       = None  # pt of jet constituent particle 3
    const_4_pt       = None  # pt of jet constituent particle 4
    const_5_pt       = None  # pt of jet constituent particle 5
    const_6_pt       = None  # pt of jet constituent particle 6
    const_7_pt       = None  # pt of jet constituent particle 7
    const_8_pt       = None  # pt of jet constituent particle 8
    const_9_pt       = None  # pt of jet constituent particle 9
    const_10_pt      = None  # pt of jet constituent particle 10
    jet_y            = None
    jet_phi          = None
    jet_rho          = None
    

    # Targets
    jet_pt_true_A    = None  # True jet pt (determined from PYTHIA jets)
    jet_pt_true_B    = None

    # Helper Variables
    event_n          = 0
    event_n_total    = 20000
    jet_n            = None  # Number of jets in an event
    jet_n_counter_A  = 0
    jet_n_counter_B  = 0
    jet_const_pt_arr = []    # Array of jet constituents and their values
    sc_correction_arr_A = []
    sc_correction_arr_B = []

    print("Preparing to collect data from tree...")
    
    # Collecting from TTree
    for event in input_tree :  
        jet_n = event.jet_n
        
        for jet in range(0, 2) :
            jet_pt_raw      = input_tree.jet_pt_raw[jet]
            jet_pt_corr     = input_tree.jet_pt_corr[jet]
            jet_mass        = input_tree.jet_mass[jet]
            jet_area        = input_tree.jet_area[jet]
            jet_const_n     = input_tree.jet_const_n[jet]
            const_pt_mean   = input_tree.const_pt_mean[jet]
            const_pt_median = input_tree.const_pt_median[jet]
            const_1_pt      = input_tree.const_1_pt[jet]
            const_2_pt      = input_tree.const_2_pt[jet]
            const_3_pt      = input_tree.const_3_pt[jet]
            const_4_pt      = input_tree.const_4_pt[jet]
            const_5_pt      = input_tree.const_5_pt[jet]
            const_6_pt      = input_tree.const_6_pt[jet]
            const_7_pt      = input_tree.const_7_pt[jet]
            const_8_pt      = input_tree.const_8_pt[jet]
            const_9_pt      = input_tree.const_9_pt[jet]
            const_10_pt     = input_tree.const_10_pt[jet]
            jet_y           = input_tree.jet_y[jet]
            jet_phi         = input_tree.jet_phi[jet]
            jet_rho         = input_tree.jet_rho[jet]
            
            jet_pt_true_A   = input_tree.jet_pt_true_pythia[jet]
            jet_pt_true_B   = input_tree.jet_pt_true_paper[jet]
            
            temp_jet_arr = [
                    jet_pt_raw,      jet_pt_corr,     const_pt_mean,        jet_area, 
                        jet_const_n,     jet_mass,   const_pt_median, jet_rho,
                    const_1_pt,      const_2_pt,      const_3_pt,      const_4_pt,
                    const_5_pt,      const_6_pt,      const_7_pt,      const_8_pt,
                    const_9_pt,      const_10_pt,     jet_y,           jet_phi,]
            
            if (jet_pt_true_A != 0.0) and (jet_pt_corr > pt_corr_min) and (jet_pt_corr < pt_corr_max) :
                
                X_values_A.append(temp_jet_arr)

                y_values_A.append(jet_pt_true_A)
                
                sc_correction_arr_A.append(jet_pt_corr)
                
                jet_n_counter_A   = jet_n_counter_A + 1
                
                if jet_n_counter_A % 10 == 0 :
                    print(f"Event: {event_n:3.0f} | Jet: {jet:2.0f} | pTraw: {jet_pt_raw:3.3f} | pTcorr: {jet_pt_corr: 3.3f} | pTtrue_A: {jet_pt_true_A: 5.3f}")
            
            if (jet_pt_true_B != 0.0) and (jet_pt_corr > pt_corr_min) and (jet_pt_corr < pt_corr_max) :
                X_values_B.append(temp_jet_arr)
                
                y_values_B.append(jet_pt_true_B)

                sc_correction_arr_B.append(jet_pt_corr)
                
                jet_n_counter_B   = jet_n_counter_B + 1

                if jet_n_counter_B % 10 == 0 :
                    print(f"Event: {event_n:3.0f} | Jet: {jet:2.0f} | pTraw: {jet_pt_raw:3.3f} | pTcorr: {jet_pt_corr: 3.3f} | pTtrue_B: {jet_pt_true_A: 5.3f}")

        event_n = event_n + 1

    print(f"All data transferred to array. Testing with {jet_n_counter_A} A-jets and {jet_n_counter_B} B-jets.\n")
    print(f"Training set A: {len(X_values_A)} / {len(y_values_A)} / {len(sc_correction_arr_A)}")
    print(f"Training set B: {len(X_values_B)} / {len(y_values_B)} / {len(sc_correction_arr_B)}")

    input_file.Close()
    print("Input file closed.")
    
    return X_values_A, y_values_A, sc_correction_arr_A, X_values_B, y_values_B, sc_correction_arr_B

Welcome to JupyROOT 6.26/06


2022-11-16 13:47:31.530795: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-16 13:47:31.703805: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-16 13:47:31.707534: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/qq/Documents/root_install/lib:/home/qq/fastjet-install/lib/
2022-11-16 13:47:31.707544: I 

In [2]:
X_train, y_train = [], []
powers = [-1, 4, 8]
train_tree_name    = "Tree_10_90_Train"
train_file_paths = []
for power in powers:
    train_directory    = "./compiled/Data/"
    train_file_name    = "ML_Prep_10_90_Train" + str(power) +".root"
    train_file_path    = train_directory + train_file_name
    train_file_paths.append(train_file_path)

for i in range(len(powers)):
    X_train_A, y_train_A, sc_corr_train_arr_A, X_train_B, y_train_B, sc_corr_train_arr_B = Build_ML_Feature_Arrays_ptTrue(
        train_file_paths[i], train_tree_name, 10.0, 90.0)
    X_train.append(X_train_A)
    y_train.append(y_train_A)

now = datetime.now()
dt_string = now.strftime("%Y/%m/%d %H:%M:%S")

print("Ready!", dt_string)

Input file accessed successfully. Output file generated.
Accessing input tree...
Input tree accessed successfully.
Preparing to collect data from tree...
Event:   0 | Jet:  0 | pTraw: 72.219 | pTcorr:  28.311 | pTtrue_A:  14.705
Event:   0 | Jet:  0 | pTraw: 72.219 | pTcorr:  28.311 | pTtrue_B:  14.609
Event: 2000 | Jet:  0 | pTraw: 66.467 | pTcorr:  8.597 | pTtrue_A:  16.938
Event: 2000 | Jet:  0 | pTraw: 66.467 | pTcorr:  8.597 | pTtrue_B:  16.746
Event: 3000 | Jet:  0 | pTraw: 63.281 | pTcorr:  14.337 | pTtrue_A:  12.846
Event: 3000 | Jet:  0 | pTraw: 63.281 | pTcorr:  14.337 | pTtrue_B:  12.678
Event: 6000 | Jet:  1 | pTraw: 52.624 | pTcorr:  14.410 | pTtrue_A:  12.432
Event: 6000 | Jet:  1 | pTraw: 52.624 | pTcorr:  14.410 | pTtrue_B:  15.034
Event: 7000 | Jet:  1 | pTraw: 72.289 | pTcorr:  13.467 | pTtrue_A:  12.241
Event: 7000 | Jet:  1 | pTraw: 72.289 | pTcorr:  13.467 | pTtrue_B:  14.999
Event: 9000 | Jet:  0 | pTraw: 71.920 | pTcorr:  8.329 | pTtrue_A:  15.030
Event: 9000 | J

Event: 98000 | Jet:  0 | pTraw: 76.359 | pTcorr:  17.901 | pTtrue_A:  11.788
Event: 98000 | Jet:  0 | pTraw: 76.359 | pTcorr:  17.901 | pTtrue_B:  11.637
Event: 99000 | Jet:  1 | pTraw: 67.402 | pTcorr:  4.897 | pTtrue_A:  10.433
Event: 100000 | Jet:  1 | pTraw: 68.743 | pTcorr:  18.067 | pTtrue_A:  15.034
Event: 100000 | Jet:  1 | pTraw: 68.743 | pTcorr:  18.067 | pTtrue_B:  18.995
Event: 101000 | Jet:  0 | pTraw: 68.879 | pTcorr:  23.690 | pTtrue_A:  17.252
Event: 101000 | Jet:  0 | pTraw: 68.879 | pTcorr:  23.690 | pTtrue_B:  16.943
Event: 102000 | Jet:  1 | pTraw: 70.891 | pTcorr:  7.393 | pTtrue_A:  10.997
Event: 102000 | Jet:  1 | pTraw: 70.891 | pTcorr:  7.393 | pTtrue_B:  12.078
Event: 104000 | Jet:  1 | pTraw: 67.448 | pTcorr:  8.737 | pTtrue_A:  16.195
Event: 104000 | Jet:  1 | pTraw: 67.448 | pTcorr:  8.737 | pTtrue_B:  19.204
Event: 105000 | Jet:  0 | pTraw: 93.530 | pTcorr:  12.342 | pTtrue_A:  13.258
Event: 105000 | Jet:  0 | pTraw: 93.530 | pTcorr:  12.342 | pTtrue_B:  1

Event: 189000 | Jet:  0 | pTraw: 79.449 | pTcorr:  7.585 | pTtrue_A:  10.537
Event: 189000 | Jet:  0 | pTraw: 79.449 | pTcorr:  7.585 | pTtrue_B:  10.376
Event: 194000 | Jet:  0 | pTraw: 67.354 | pTcorr:  17.535 | pTtrue_A:  11.861
Event: 194000 | Jet:  0 | pTraw: 67.354 | pTcorr:  17.535 | pTtrue_B:  11.740
Event: 196000 | Jet:  0 | pTraw: 82.234 | pTcorr:  17.715 | pTtrue_A:  25.662
Event: 196000 | Jet:  0 | pTraw: 82.234 | pTcorr:  17.715 | pTtrue_B:  25.423
Event: 197000 | Jet:  0 | pTraw: 77.303 | pTcorr:  22.134 | pTtrue_A:  12.309
Event: 197000 | Jet:  0 | pTraw: 77.303 | pTcorr:  22.134 | pTtrue_B:  12.115
Event: 200000 | Jet:  0 | pTraw: 74.678 | pTcorr:  14.611 | pTtrue_A:  12.290
Event: 200000 | Jet:  0 | pTraw: 74.678 | pTcorr:  14.611 | pTtrue_B:  12.232
Event: 201000 | Jet:  0 | pTraw: 80.156 | pTcorr:  17.982 | pTtrue_A:  11.676
Event: 201000 | Jet:  0 | pTraw: 80.156 | pTcorr:  17.982 | pTtrue_B:  11.546
Event: 202000 | Jet:  0 | pTraw: 81.889 | pTcorr:  17.659 | pTtrue

Event: 23000 | Jet:  1 | pTraw: 104.904 | pTcorr:  60.840 | pTtrue_A:  56.576
Event: 24000 | Jet:  0 | pTraw: 106.538 | pTcorr:  52.347 | pTtrue_A:  41.586
Event: 24000 | Jet:  0 | pTraw: 106.538 | pTcorr:  52.347 | pTtrue_B:  41.134
Event: 24000 | Jet:  1 | pTraw: 75.899 | pTcorr:  37.964 | pTtrue_A:  35.296
Event: 24000 | Jet:  1 | pTraw: 75.899 | pTcorr:  37.964 | pTtrue_B:  31.842
Event: 25000 | Jet:  0 | pTraw: 80.900 | pTcorr:  16.710 | pTtrue_A:  23.859
Event: 25000 | Jet:  0 | pTraw: 80.900 | pTcorr:  16.710 | pTtrue_B:  23.638
Event: 26000 | Jet:  0 | pTraw: 124.903 | pTcorr:  56.337 | pTtrue_A:  59.129
Event: 26000 | Jet:  0 | pTraw: 124.903 | pTcorr:  56.337 | pTtrue_B:  58.471
Event: 27000 | Jet:  0 | pTraw: 86.029 | pTcorr:  24.007 | pTtrue_A:  41.139
Event: 27000 | Jet:  0 | pTraw: 86.029 | pTcorr:  24.007 | pTtrue_B:  40.861
Event: 27000 | Jet:  1 | pTraw: 78.915 | pTcorr:  10.812 | pTtrue_A:  20.852
Event: 27000 | Jet:  1 | pTraw: 78.915 | pTcorr:  10.812 | pTtrue_B:  1

Event: 75000 | Jet:  0 | pTraw: 69.971 | pTcorr:  21.575 | pTtrue_A:  39.299
Event: 75000 | Jet:  0 | pTraw: 69.971 | pTcorr:  21.575 | pTtrue_B:  39.083
Event: 75000 | Jet:  1 | pTraw: 66.141 | pTcorr:  7.224 | pTtrue_A:  15.095
Event: 75000 | Jet:  1 | pTraw: 66.141 | pTcorr:  7.224 | pTtrue_B:  12.056
Event: 76000 | Jet:  0 | pTraw: 75.515 | pTcorr:  20.218 | pTtrue_A:  17.352
Event: 76000 | Jet:  0 | pTraw: 75.515 | pTcorr:  20.218 | pTtrue_B:  17.186
Event: 77000 | Jet:  0 | pTraw: 63.706 | pTcorr:  12.109 | pTtrue_A:  15.113
Event: 77000 | Jet:  0 | pTraw: 63.706 | pTcorr:  12.109 | pTtrue_B:  14.902
Event: 78000 | Jet:  0 | pTraw: 68.702 | pTcorr:  12.172 | pTtrue_A:  14.149
Event: 78000 | Jet:  0 | pTraw: 68.702 | pTcorr:  12.172 | pTtrue_B:  14.032
Event: 79000 | Jet:  1 | pTraw: 72.267 | pTcorr:  24.749 | pTtrue_A:  26.377
Event: 79000 | Jet:  1 | pTraw: 72.267 | pTcorr:  24.749 | pTtrue_B:  29.693
Event: 80000 | Jet:  0 | pTraw: 71.855 | pTcorr:  25.083 | pTtrue_A:  17.152
E

Event: 38000 | Jet:  1 | pTraw: 229.918 | pTcorr:  174.797 | pTtrue_B:  85.541
Event: 39000 | Jet:  1 | pTraw: 70.567 | pTcorr:  25.598 | pTtrue_A:  27.972
Event: 39000 | Jet:  1 | pTraw: 70.567 | pTcorr:  25.598 | pTtrue_B:  31.729
Event: 40000 | Jet:  1 | pTraw: 80.200 | pTcorr:  34.795 | pTtrue_A:  30.732
Event: 40000 | Jet:  1 | pTraw: 80.200 | pTcorr:  34.795 | pTtrue_B:  33.552
Event: 43000 | Jet:  0 | pTraw: 149.512 | pTcorr:  86.585 | pTtrue_A:  66.409
Event: 43000 | Jet:  0 | pTraw: 149.512 | pTcorr:  86.585 | pTtrue_B:  65.721
Event: 44000 | Jet:  0 | pTraw: 112.652 | pTcorr:  50.928 | pTtrue_A:  47.507
Event: 44000 | Jet:  1 | pTraw: 82.061 | pTcorr:  33.650 | pTtrue_A:  29.257
Event: 45000 | Jet:  1 | pTraw: 124.170 | pTcorr:  64.347 | pTtrue_A:  73.350
Event: 46000 | Jet:  1 | pTraw: 136.363 | pTcorr:  72.513 | pTtrue_A:  82.718
Event: 46000 | Jet:  1 | pTraw: 136.363 | pTcorr:  72.513 | pTtrue_B:  27.385
Event: 48000 | Jet:  0 | pTraw: 125.815 | pTcorr:  67.295 | pTtrue_A

In [3]:
output_dir = "./compiled/Data"
output_fil = "results_10_90"
output_ext = ".root"
newX = []
newY = []

In [4]:
# for i in range(len(y_train[2])):
#     if y_train[6][i] > 60:
#         newX.append(X_train[6][i])
#         newY.append(y_train[6][i])
#     elif y_train[6][i] > 50:
#         if not np.random.uniform(0,1) < 0.2:
#             newX.append(X_train[6][i])
#             newY.append(y_train[6][i])
#     elif y_train[6][i] > 40:
#         if not np.random.uniform(0,1) < 0.25:
#             newX.append(X_train[6][i])
#             newY.append(y_train[6][i])
#     elif y_train[6][i] > 30:
#         if not np.random.uniform(0,1) < 0.45:
#             newX.append(X_train[6][i])
#             newY.append(y_train[6][i])
#     elif y_train[6][i] > 20:
#         if not np.random.uniform(0,1) < 0.625:
#             newX.append(X_train[6][i])
#             newY.append(y_train[6][i])
#     else:
#         if not np.random.uniform(0,1) < 0.82:
#             newX.append(X_train[6][i])
#             newY.append(y_train[6][i])
            

In [5]:
#counts,bins = np.histogram(newY, bins=9)
#plt.hist(bins[:-1], bins, weights=counts, range=(10,90))

def test(X_test, y_test, indices, model, i, l, k):
    y_pred = model.predict(X_test)
    xmin = -49.5
    xmax = 50.5
    bins = 100
    outf = ROOT.TFile.Open(output_directory + output_fil + "train_ptbias" + str(l) + "test_ptbias" + str(powers[k]) + model.__class__.__name__ + str(len(indices)) + "Features_setnumber"+ str(group)+ output_ext, "RECREATE")
    outt = ROOT.TTree(test_tree_names[0] + model.__class__.__name__, "tree")
    csvfile = test_tree_names[0][5:-5] + model.__class__.__name__ + ".csv"
    f = open(csvfile, "w")
    deltahist = ROOT.TH1D("hist", "Jet p_{T} Delta " + str(len(indices)) + " Feature(s) Test " + test_tree_names[0][5:-5] + " GeV (Falling) Test 10_90 GeV (Falling) Train", bins, xmin, xmax)
    deltahist.GetXaxis().SetTitle("p_{T true} - p_{T predicted}")
    deltahist.GetYaxis().SetTitle("Counts")
    deltahist.GetXaxis().SetRangeUser(-50, 50)
    deltahist.GetYaxis().SetRangeUser(0,750)
    deltahist.SetMarkerStyle(2)
    deltahist.SetMarkerColor(q)
    for j in range(len(y_test)):
        deltahist.Fill((y_pred[j] - y_test[j]))
        f.write(str((y_test[j] - y_pred[j])) + "\n")
    deltahist.SetDirectory(0)
    outf.cd()
    deltahist.Write()
    outf.Write()
    outf.Close()
    f.close()
    return 0

In [6]:
#y_train.append(newY)
#X_train.append(newX)

def fill_schist(corvals, truevals, indices, i, l, k):
    xmin = -49.5
    xmax = 50.5
    bins = 100
    outf = ROOT.TFile.Open(output_directory + output_fil + "train_ptbias" + str(l) + "test_ptbias" + str(powers[k]) +"corrected" + str(len(indices)) + "Features_setnumber"+ str(group) + output_ext, "RECREATE")
    outt = ROOT.TTree(test_tree_names[0][:-5] + " corrected", "tree")
    deltahist = ROOT.TH1D("hist", "Jet p_{T} Delta " + str(len(indices)) + " Feature(s) Test " + test_tree_names[0][5:-5] + " GeV (Falling) Test 10_90 GeV (Falling) Test", bins, xmin, xmax)
    deltahist.GetXaxis().SetTitle("p_{T true} - p_{T predicted}")
    deltahist.GetYaxis().SetTitle("Counts")
    deltahist.GetXaxis().SetRangeUser(-50, 50)
    deltahist.GetYaxis().SetRangeUser(0,750)
    deltahist.SetMarkerStyle(2)
    deltahist.SetMarkerColor(5)
    csvfile = "Correction" + str(len(indices))+ ".csv"
    f = open(csvfile, "w")
    for j in range(len(corvals)):
        deltahist.Fill((corvals[j][0] - truevals[j]))
        f.write(str((corvals[j][0]-truevals[j])))
    deltahist.SetDirectory(0)
    outf.cd()
    deltahist.Write()
    outf.Write()
    outf.Close()
    f.close()
    return 0

In [7]:
def evaluate_model(model, indices, X_test, y_test, k, p, l):
    newX_train = []
    for i in range(len(X_train[l])):
        ph = []
        for j in range(len(X_train[l][0])):
            if j in indices:
                ph.append(X_train[l][i][j])
        newX_train.append(ph)
    model.fit(newX_train, y_train[l])
    if(q%5 == 0):
        print(model.coef_)
    elif(q%5 == 1):
        print(model.feature_importances_)
    newX_test = []
    for i in range(len(X_test)):
        ph = []
        for j in range(len(X_test[i])):
            if j in indices:
                ph.append(X_test[i][j])
        newX_test.append(ph)
    test(newX_test, y_test, indices, model, k, p, l)

In [16]:
powers = [-1,4,8]
test_directory     = "./compiled/Data/"
test_file_names    = []
group = 2
ptrange = [[40,60],[40,60],[48,52],[48,52]]
#for power in powers:
test_file_names.append("ML_Prep_40_60_Test" + str(-1) + ".root")
# test_file_names    = ["ML_Prep_40_60_Test.root"]
test_tree_names    = ["Tree_40_60_Test"]
# test_tree_names    = ["Tree_40_60_Test"]
test_file_paths    = [test_directory + test_file_names[i] for i in range(len(test_file_names))]
print(test_file_paths)
output_directory   = "./compiled/Data/"
output_file_name   = "ML_Results_10_90.root"
output_file_path   = output_directory + output_file_name
X_test, y_test = [], []
for i in range(len(test_file_names)):
    X, y, _,_,_,_ = Build_ML_Feature_Arrays_ptTrue(test_file_paths[i], test_tree_names[0], ptrange[group][0], ptrange[group][1])
    X_test.append(X)
    y_test.append(y)


['./compiled/Data/ML_Prep_40_60_Test-1.root']
Input file accessed successfully. Output file generated.
Accessing input tree...
Input tree accessed successfully.
Preparing to collect data from tree...
Event: 7000 | Jet:  1 | pTraw: 111.728 | pTcorr:  64.062 | pTtrue_A:  48.762
Event: 10000 | Jet:  0 | pTraw: 107.886 | pTcorr:  43.912 | pTtrue_A:  49.883
Event: 10000 | Jet:  0 | pTraw: 107.886 | pTcorr:  43.912 | pTtrue_B:  49.525
Event: 17000 | Jet:  0 | pTraw: 111.175 | pTcorr:  51.028 | pTtrue_A:  50.398
Event: 17000 | Jet:  0 | pTraw: 111.175 | pTcorr:  51.028 | pTtrue_B:  49.995
Event: 21000 | Jet:  0 | pTraw: 125.443 | pTcorr:  61.040 | pTtrue_A:  49.319
Event: 21000 | Jet:  0 | pTraw: 125.443 | pTcorr:  61.040 | pTtrue_B:  48.916
Event: 32000 | Jet:  0 | pTraw: 114.833 | pTcorr:  69.388 | pTtrue_A:  51.800
Event: 32000 | Jet:  0 | pTraw: 114.833 | pTcorr:  69.388 | pTtrue_B:  51.602
Event: 36000 | Jet:  0 | pTraw: 103.543 | pTcorr:  45.680 | pTtrue_A:  48.435
Event: 41000 | Jet:  

In [17]:
indices = [[0,1,2,3,4,5,6,7,8,9,10,11]]
indices.append([0,3,7])
indices.append([0])
lr = LinearRegression()
rf = RandomForestRegressor()
nn = MLPRegressor()
el = ElasticNet()
# nn = [keras.Sequential(),keras.Sequential(),keras.Sequential()]
# nn[0].add(layers.Dense(16, input_shape=(12,)))#, activation='tanh'))
# nn[0].add(layers.Dense(8))#, activation='tanh'))
# nn[0].add(layers.Dense(10))
# nn[0].add(layers.Dense(5))
# nn[0].add(layers.Dense(1, activation='relu'))
# nn[0].compile(optimizer='sgd', loss='mse', metrics=[r2])
# nn[1].add(layers.Dense(3, input_shape=(3,), activation='tanh'))
# nn[1].add(layers.Dense(1, activation='relu'))
# nn[1].compile(optimizer='sgd', loss='mse', metrics=[r2])
# nn[2].add(layers.Dense(1, input_shape=(1,), activation='tanh'))
# nn[2].compile(optimizer='sgd', loss='mse', metrics=[r2])
q=0
for j in range(len(X_train)):
    for i in range(len(indices)):
        for k in range(len(X_test)):
            #evaluate_model(nn[i], indices[i], X_test[k], y_test[k], i, powers[j], k)
            evaluate_model(lr, indices[i], X_test[k], y_test[k],i, powers[j], k)
            q += 1
            evaluate_model(rf, indices[i], X_test[k], y_test[k],i, powers[j],k)
            q += 1
            evaluate_model(nn, indices[i], X_test[k], y_test[k],i, powers[j],k)
            q += 1
            fill_schist(X_test[k], y_test[k], indices[i], i, powers[j], k)
            q += 1
            evaluate_model(el ,indices[i], X_test[k], y_test[k], i, powers[j],k)
            q += 1
            print("Train set " + str(j) + ", " + str(len(indices[i])) + " features, test set " + str(k) + "done.")

[-1.11526676e-02  9.54892239e-01  6.73211712e+00 -1.86466209e+00
 -2.15150774e-01  8.49928674e+00 -1.71993582e+01 -5.09783808e-02
  8.40203053e-03  3.44843987e-02  3.01143642e-02  7.62173173e-03]
[0.13090099 0.10193207 0.01520038 0.0664105  0.04217834 0.42622431
 0.07937671 0.02753748 0.02729403 0.02769641 0.02736584 0.02788295]
Train set 0, 12 features, test set 0done.
[ 0.49761173 -0.85281026  0.14477004]
[0.55862348 0.23545696 0.20591956]
Train set 0, 3 features, test set 0done.
[0.34672614]
[1.]
Train set 0, 1 features, test set 0done.
[-1.11526676e-02  9.54892239e-01  6.73211712e+00 -1.86466209e+00
 -2.15150774e-01  8.49928674e+00 -1.71993582e+01 -5.09783808e-02
  8.40203053e-03  3.44843987e-02  3.01143642e-02  7.62173173e-03]
[0.13304133 0.09923038 0.01514233 0.06619716 0.04148485 0.42820448
 0.07909925 0.02765986 0.02733254 0.02769687 0.02722538 0.02768556]
Train set 1, 12 features, test set 0done.
[ 0.49761173 -0.85281026  0.14477004]
[0.5594601  0.23452826 0.20601164]
Train se